## Analysis of Tests done in India and Comparission with South Korea
<hr>
**Note:** I am working on more on testing data and comparisions insights and will adding soon.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
import folium
import branca
from datetime import datetime, timedelta,date
import plotly.express as px
import json, requests

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# COVID-19 : INDIA ANALYSIS
<hr>
### Introduction
![corona](https://images.unsplash.com/photo-1583324113626-70df0f4deaab?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1000&q=80)
> Coronavirus is a family of viruses that can cause illness, which can vary from common cold and cough to sometimes more severe disease. Middle East Respiratory Syndrome (MERS-CoV) and Severe Acute Respiratory Syndrome (SARS-CoV) were such severe cases with the world already has faced.
SARS-CoV-2 (n-coronavirus) is the new virus of the coronavirus family, which first discovered in 2019, which has not been identified in humans before. It is a contiguous virus which started from Wuhan in December 2019. Which later declared as Pandemic by WHO due to high rate spreads throughout the world. Currently (on the date 5 May 2020), this leads to a total of 251K+ Deaths across the globe.
Pandemic is spreading all over the world; it becomes more important to understand about this spread. This NoteBook is an effort to analyze the cumulative data of confirmed, deaths, and recovered cases over time. In this notebook, the main focus is to analyze the spread trend of this virus all over the world.

> In India the first case was reported on 28 Jan, 2020, and as of now (5 May, 2020) more than 40K+ cases have been reported. More than 1300+ peoples have been died so far.

<hr>



<hr>

### Sources
* [India Dashboard-Covid19](https://www.covid19india.org/)


### Dataset:
This dataset is provided by https://api.rootnet.in/

In [ ]:
df_india_test = pd.io.json.json_normalize(requests.get('https://api.rootnet.in/covid19-in/stats/testing/history').json()['data']).rename(columns = {"totalIndividualsTested":"c_individualtest","totalPositiveCases":"c_positive","totalSamplesTested":"c_tests"})

In [ ]:
df_india_test["p2t_ratio"]= np.round(100*df_india_test["c_positive"]/df_india_test["c_tests"],2)
df_india_test["positive"] = df_india_test["c_positive"].diff()
df_india_test["tests"] = df_india_test["c_tests"].diff()
df_india_test["p2t_ratio"]= np.round(100*df_india_test["positive"]/df_india_test["tests"],2)
df_india_test = df_india_test[1:]

In [ ]:
f = plt.figure(figsize=(10,5))
f.add_subplot(111)

plt.axes(axisbelow=True)
plt.bar(df_india_test["day"],df_india_test["tests"].values[:],color="darkcyan",label="Test (Negetive)"+str())
plt.bar(df_india_test["day"],df_india_test["positive"].values[:],bottom=df_india_test["tests"].values[:]-df_india_test["positive"].values[:],color="Red",label="Test (Positive)")
plt.tick_params(size=5,labelsize = 13)
plt.tick_params(axis="x",size=5,labelsize = 13,labelrotation=60 )
plt.xlabel("Date",fontsize=18)
plt.ylabel("Test Results",fontsize=18)

plt.title("Tests in India",fontsize=20)
plt.grid(alpha=0.3)
plt.legend()
plt.savefig('Test_India.png')

## Number of Test in India till date

In [ ]:
df_india_test["c_tests"][-1:].values[0]

## Test Conducted per Million People

In [ ]:
np.round(1000000*df_india_test["c_tests"][-1:].values[0]/1300000000,2)

This is far less than the tests happening in South Korea, Singapore and Other countries. Here is a comparison table for India and South Korea <br>**(Last Update: 2 May 2020)**
<table>
    <tr>
        <th style = "text-align:Left">Parameters</th>
        <th>India</th>
        <th>South Korea</th>
    </tr>
    <tr>
        <th style = "text-align:Left">Population</th>
        <td>130 Cr</td>
        <td>5.15 Cr</td>
    </tr>
    <tr>
        <th style = "text-align:Left">First Case Reported</th>
        <td>28 Jan 2020</td>
        <td>23 Jan 2020</td>
    </tr>
    <tr>
        <th style = "text-align:Left">Total Tests</th>
        <td>976363</td>
        <td>627,562</td>
    </tr>
    <tr>
        <th style = "text-align:Left">Tests Per Million Peoples</th>
        <td>751</td>
        <td>12186</td>
    </tr>
    <tr>
        <th style = "text-align:Left">Positives</th>
        <td>1250</td>
        <td>9661</td>
    </tr>
    <tr>
        <th style = "text-align:Left">No. Positives in 100 Tests</th>
        <td>4.07</td>
        <td>1.72</td>
    </tr>
    <tr>
        <th style = "text-align:Left">Current Mortality Rate</th>
        <td>3.33</td>
        <td>2.32</td>
    </tr>
</table><br>
From this table, this can be concluded that India is far behind in terms of testing, and testing is very important to keep track of COVID19 spread and also to quarantine those people or isolate them to control the spread of COVID19 as done by South Korea.

## COVID19 Cases in India
<hr>

In [ ]:
india_data_json = requests.get('https://api.rootnet.in/covid19-in/unofficial/covid19india.org/statewise').json()
df_india = pd.io.json.json_normalize(india_data_json['data']['statewise'])
df_india = df_india.set_index("state")
df_india

In [ ]:
total = df_india.sum()
total.name = "Total"
df_t = pd.DataFrame(total,dtype=float).transpose()
df_t["Mortality Rate (per 100)"] = np.round(100*df_t["deaths"]/df_t["confirmed"],2)
df_t.style.background_gradient(cmap='Purples',axis=1)

In [ ]:
df_india["Mortality Rate (per 100)"]= np.round(np.nan_to_num(100*df_india["deaths"]/df_india["confirmed"]),2)
df_india.style.background_gradient(cmap='Blues',subset=["confirmed"])\
                        .background_gradient(cmap='Reds',subset=["deaths"])\
                        .background_gradient(cmap='Greens',subset=["recovered"])\
                        .background_gradient(cmap='Purples',subset=["active"])\
                        .background_gradient(cmap='YlOrBr',subset=["Mortality Rate (per 100)"])

In [ ]:
df_india[df_india['deaths'] > 0].style.background_gradient(cmap='Blues',subset=["confirmed"])\
                        .background_gradient(cmap='Reds',subset=["deaths"])\
                        .background_gradient(cmap='Greens',subset=["recovered"])\
                        .background_gradient(cmap='Purples',subset=["active"])\
                        .background_gradient(cmap='YlOrBr',subset=["Mortality Rate (per 100)"])

## 10 Most Affected States

In [ ]:
f = plt.figure(figsize=(10,5))
f.add_subplot(111)

plt.axes(axisbelow=True)
plt.barh(df_india.sort_values('confirmed')["confirmed"].index[-10:],df_india.sort_values('confirmed')["confirmed"].values[-10:],color="darkcyan")
plt.tick_params(size=5,labelsize = 13)
plt.xlabel("Confirmed Cases",fontsize=18)
plt.title("Top 10 States: India (Confirmed Cases)",fontsize=20)
plt.grid(alpha=0.3)
plt.savefig('Top 10 States_India (Confirmed Cases).png')

In [ ]:
f = plt.figure(figsize=(10,5))
f.add_subplot(111)

plt.axes(axisbelow=True)
plt.barh(df_india.sort_values('deaths')["deaths"].index[-10:],df_india.sort_values('deaths')["deaths"].values[-10:],color="crimson")
plt.tick_params(size=5,labelsize = 13)
plt.xlabel("Deaths",fontsize=18)
plt.title("Top 10 States: India (Deaths Cases)",fontsize=20)
plt.grid(alpha=0.3)
plt.savefig('Top 10 States_India (Deaths Cases).png')

## Correlation

In [ ]:
df_india.corr().style.background_gradient(cmap='Reds')